# 1. Switch Health

In [ ]:
#     result:    
#         String orgID;
#         String siteID;
#         String deviceID; // device = "switch"
#         String deviceModel;
#         String deviceVersion;
#         Long timeStamp;
#         Integer duration = 60;

#         String clientID =""; // "interface_name for throughput; Chassis_index
#         String metric = "";  // switch-throughput, switch-health,  switch-
#         String classifier = "";
#         Float value;

# Load Data

In [ ]:
# Data Flattening
import numpy as np
import pandas as pd

import datetime
from datetime import datetime

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, MapType, LongType
from pyspark.sql.functions import lit, udf, size, avg, min as min_, max as max_, sum as sum_, count, countDistinct, arrays_zip, col, mean, stddev, struct, explode, explode_outer, unix_timestamp, sum as sum_
from operator import itemgetter
import json

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import Bucketizer

from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import StandardScaler

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, row_number, dense_rank, lead, lag, rank
from pyspark.sql.window import Window

from pyspark.ml.classification import LogisticRegressionModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import collections
from collections import deque
import redis
import json


In [ ]:
date = '2019-11-07'
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats-analytics/oc-stats-analytics-staging/dt='+date+'/*')

dfeng = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                 explode_outer('chassis_routing_engines').alias('eng'))

dfeng = dfeng.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                  col('eng.module_index').alias('module_index'),
                     col('eng.is_master').alias('is_master'),
                     col('eng.status').alias('status'),
                     col('eng.temp_celsius').alias('temp_celsius'),
                     col('eng.memory_total_mb').alias('memory_total_mb'),
                     col('eng.memory_in_use_mb').alias('memory_in_use_mb'),
                     col('eng.cpu_user').alias('cpu_user'),
                     col('eng.cpu_background').alias('cpu_background'),
                     col('eng.cpu_system').alias('cpu_system'),
                     col('eng.cpu_interrupt').alias('cpu_interrupt'),
                     col('eng.cpu_idle').alias('cpu_idle'),
                     col('eng.cputemp_celsius').alias('cputemp_celsius'),
                     col('eng.model').alias('eng_model'),
                     col('eng.last_reboot_reason').alias('last_reboot_reason'),
                     col('eng.load_avg').alias('load_avg'))
                     
dfeng = dfeng.withColumn('load_avg1', dfeng.load_avg[0])  
# memorypct = udf(lambda x: x[0]/x[1], FloatType())
# dfeng = dfeng.withColumn('memory_pct', memorypct(struct('memory_in_use_mb','memory_total_mb')))
dfeng = dfeng.filter(col('delta')==True)
dfeng.write.parquet('s3://mist-data-science-dev/jing/switch/engines'+date)

# Built baseline and upperlimit using historical data


In [ ]:
def get_memorypct(dfeng):
    # calculate memory %
    memorypct = udf(lambda x: (x[0]/x[1])*100, FloatType())
    dfeng = dfeng.withColumn('memory_pct', memorypct(struct('memory_in_use_mb','memory_total_mb')))

    # filter delts=T
    dfeng = dfeng.filter(col('delta')==True)
    return dfeng

def get_baseline(df0):
    # Build Baseline
    df1 = df0.groupby('mac', 'module_index').agg(avg('temp_celsius').alias('avg_temp'), stddev('temp_celsius').alias('std_temp') \
                                                ,avg('memory_pct').alias('avg_memorypct'), stddev('memory_pct').alias('std_memorypct') \
                                                ,avg('memory_in_use_mb').alias('avg_memory'), stddev('memory_in_use_mb').alias('std_memory'))
    df2 = df0.groupby('model').agg(avg('memory_pct').alias('avg_memorypct_model'), stddev('memory_pct').alias('std_memorypct_model'), \
                                                       avg('memory_in_use_mb').alias('avg_memory_model'), stddev('memory_in_use_mb').alias('std_memory_model'))
    df1 = df1.withColumnRenamed('mac','mac1') \
            .withColumnRenamed('module_index','module_index1')
    df2 = df2.withColumnRenamed('model','model2')
    
    df0 = df0.join(df1, (df0.mac==df1.mac1)&(df0.module_index==df1.module_index1), 'inner') \
                .join(df2, (df0.model==df2.model2), 'inner')
    
    return df0

def get_upper_limit(df0):
    upper = udf(lambda x: x[0]+3*x[1], FloatType())
    df0 = df0.withColumn('upper_temp', upper(struct('avg_temp', 'std_temp'))) \
                .withColumn('upper_memorypct', upper(struct('avg_memorypct', 'std_memorypct'))) \
                .withColumn('upper_memory', upper(struct('avg_memory', 'std_memory'))) \
                .withColumn('upper_memorypct_model', upper(struct('avg_memorypct_model', 'std_memorypct_model'))) \
                .withColumn('upper_memory_model', upper(struct('avg_memory_model', 'std_memory_model'))) 
                

    df0 = df0.select('mac','module_index','org_id','site_id','model','firmware_version','upper_temp','upper_memory','upper_memorypct','upper_memory_model','upper_memorypct_model')
    return df0

def main(dfeng):
    dfeng = get_memorypct(dfeng)
    df0 = get_baseline(dfeng)
    df0 = get_upper_limit(df0).distinct()
    df0.write.parquet('s3://mist-data-science-dev/jing/switch_health/baseline_wmodel')

if __name__ == "__main__":
    # input dfeng is flattened engine data
    dfeng = dfeng.select('mac','module_index','when','model','temp_celsius','memory_in_use_mb','memory_total_mb','cpu_idle', 'org_id','site_id','model','firmware_version')    

    main(dfeng)
    

# Scoring new data 

In [80]:
 
def get_memorypct(dfeng):
    # calculate memory %
    memorypct = udf(lambda x: (x[0]/x[1])*100, FloatType())
    dfeng = dfeng.withColumn('memory_pct', memorypct(struct('memory_in_use_mb','memory_total_mb')))

    # filter delts=T
    dfeng = dfeng.filter(col('delta')==True)
    return dfeng

def smooth(list, degree=5):
    window = degree*2-1
    weight = numpy.array([1.0]*window)
    weightGauss = []
    for i in range(window):
        i = i-degree+1
        frac = i/float(window)
        gauss = 1/(numpy.exp((4*(frac))**2))
        weightGauss.append(gauss)
    weight = numpy.array(weightGauss)*weight
    smoothed = [0.0]*(len(list)-window)
    for i in range(len(smoothed)):
        smoothed[i] = (sum(numpy.array(list[i:i+window])*weight)/sum(weight)).item()
    return smoothed 

def smoothwhen(list, degree=5):
    window = degree*2-1
    return list[:(len(list)-window)]

def get_smooth(df):
    df = df.groupby('mac','module_index').agg(collect_list('memory_pct').alias('lst_memorypct'), \
                                              collect_list('memory_in_use_mb').alias('lst_memory'), \
                                             collect_list('cpu_idle').alias('lst_cpu'), \
                                             collect_list('temp_celsius').alias('lst_temp'),
                                             collect_list('when').alias('lst_when'))
    smooth_udf = udf(smooth, ArrayType(FloatType()))
    smoothwhen_udf = udf(smoothwhen, ArrayType(LongType()))
    df = df.withColumn('s_temp', smooth_udf('lst_temp')) \
            .withColumn('s_cpu', smooth_udf('lst_cpu')) \
            .withColumn('s_memory', smooth_udf('lst_memory')) \
            .withColumn('s_memorypct', smooth_udf('lst_memorypct')) \
            .withColumn('s_when', smoothwhen_udf('lst_when'))
    df = df.drop('lst_memory','lst_temp','lst_cpu', 'lst_when', 'lst_memorypct')     
    df = df.withColumn('smooth_cols', arrays_zip('s_temp','s_cpu','s_memory', 's_memorypct', 's_when')) \
            .withColumn('smooth', explode('smooth_cols')) \
            .select('mac','module_index', \
                    col('smooth.s_temp').alias('stemp'), \
                    col('smooth.s_cpu').alias('scpu'), \
                    col('smooth.s_memory').alias('smemory'), \
                    col('smooth.s_memorypct').alias('smemorypct'), \
                    col('smooth.s_when').alias('swhen'))
    return df

def get_classifier(df):
    
    def AD_func(x):
        if x[0] > x[1]: return 1
        else: return 0
    def alarm_temp(x):
        if x > 70: return 1
        else: return 0
#     def alarm_memory(x):
#         if x > 90: return 1
#         else: return 0
    def alarm_cpu(x):
        if x < 10: return 1  # 4% of all records
        else: return 0  

    AD_func = udf(AD_func, IntegerType())    
    alarm_temp = udf(alarm_temp, IntegerType())
#     alarm_memory = udf(alarm_memory, IntegerType())
    alarm_cpu = udf(alarm_cpu, IntegerType())

    df = df.withColumn('AD_temp', AD_func(struct('stemp', 'upper_temp')))
    df = df.withColumn('AD_memory', AD_func(struct('smemory', 'upper_memory')))
    df = df.withColumn('AD_memorypct', AD_func(struct('smemorypct', 'upper_memorypct')))
    df = df.withColumn('AD_memory_model', AD_func(struct('smemory', 'upper_memory_model')))
    df = df.withColumn('AD_memorypct_model', AD_func(struct('smemorypct', 'upper_memorypct_model')))
    df = df.withColumn('alarm_temp', alarm_temp('stemp'))
#     df = df.withColumn('alarm_memory', alarm_memory('smemory'))
    df = df.withColumn('alarm_cpu', alarm_cpu('scpu'))
    
    
    def detection(x):
        root = np.sum([i for i in x]).item()        
        if (x[2] == 1) & (x[0] == 1): 
            root -= 1
        if (x[3] == 1) & (x[1] == 1):
            root -= 1
        return root
    detection = udf(detection, IntegerType())
    df = df.withColumn('detection', detection(struct('AD_temp','AD_memory','alarm_temp','AD_memory','AD_memorypct','AD_memory_model','AD_memorypct_model','alarm_cpu'))) 
    return df

def get_histogram(df):
    splits = [-float('inf'),0,20,40,60,80,100,float('inf')]
    tempbucket = Bucketizer(splits=splits, inputCol="stemp", outputCol="tempbucket") 
    tempbucket = tempbucket.transform(df) 
    temphist = tempbucket.groupby('mac','module_index','tempbucket').count()
    memorybucket = Bucketizer(splits=splits, inputCol="smemory", outputCol="memorybucket") 
    memorybucket = memorybucket.transform(df) 
    memoryhist = memorybucket.groupby('mac','module_index','memorybucket').count()
    cpubucket = Bucketizer(splits=splits, inputCol="scpu", outputCol="cpubucket") 
    cpubucket = cpubucket.transform(df) 
    cpuhist = cpubucket.groupby('mac','module_index','cpubucket').count()
    return temphist, memoryhist, cpuhist
    
def __main__(dfeng):
    dfin = dfeng.select('mac','module_index','when','temp_celsius','memory_in_use_mb','memory_total_mb','cpu_idle', 'org_id','site_id','model','firmware_version')
    df = get_memorypct(dfin)
    df = get_smooth(df)
    
    # add baseline to df
    df0 = spark.read.parquet('s3://mist-data-science-dev/jing/switch_health/baseline')    
    df = df.join(df0, (df.mac == df0.mac1)&(df.module_index == df0.module_index1), 'left') \
            .drop('mac1','module_index1')
    
    classifier = get_classifier(df) 
    classifier.toPandas().to_csv('classifier.csv')
    classifier.write.parquet('s3://mist-data-science-dev/jing/switch_health/classifier')
    
    temphist, memoryhist, cpuhist = get_histogram(df)
    temphist.write.parquet('s3://mist-data-science-dev/jing/switch_health/temphist')
    memoryhist.write.parquet('s3://mist-data-science-dev/jing/switch_health/memoryhist')
    cpuhist.write.parquet('s3://mist-data-science-dev/jing/switch_health/cpuhist')
    temphist.sort('tempbucket').toPandas().to_csv('temphist.csv')
    memoryhist.sort('memorybucket').toPandas().to_csv('memoryhist.csv')
    cpuhist.sort('cpubucket').toPandas().to_csv('cpuhist.csv')
    
if __name__ == "__main__":
    __main__(dfeng)

IndentationError: expected an indented block (<ipython-input-80-d380d8604b2b>, line 68)

# ------END-----

---------------------------

# 2. UPLINK #

# Load Data

In [ ]:
# Data Flattening
import numpy as np
import pandas as pd

import datetime
from datetime import datetime

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, MapType, LongType
from pyspark.sql.functions import lit, udf, size, avg, min as min_, max as max_, sum as sum_, count, countDistinct, arrays_zip, col, mean, stddev, struct, explode, explode_outer, unix_timestamp, sum as sum_
from operator import itemgetter
import json

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import Bucketizer

from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import StandardScaler

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, row_number, dense_rank, lead, lag, rank
from pyspark.sql.window import Window

from pyspark.ml.classification import LogisticRegressionModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import collections
from collections import deque
import redis
import json


In [ ]:
#flatten interface
date = '2019-11-08'
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats-analytics/oc-stats-analytics-staging/dt='+date+'/*')

dfin = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                 explode_outer('interfaces').alias('interfaces'))

dfin = dfin.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                  col('interfaces.name').alias('name'),
                   col('interfaces.link').alias('link'),
                   col('interfaces.full_duplex').alias('full_duplex'),
                   col('interfaces.mbps').alias('mbps'),
                   col('interfaces.mtu').alias('mtu'),
                   col('interfaces.address').alias('address'),
                   col('interfaces.admin_status').alias('admin_status'),
                   col('interfaces.last_flapped').alias('last_flapped'),
                   col('interfaces.errors').alias('errors'),
                   col('interfaces.poe_enabled').alias('poe_enabled'),
                   col('interfaces.poe_status').alias('poe_status'),
                   col('interfaces.poe_power_limit').alias('poe_power_limit'),
                   col('interfaces.poe_power').alias('poe_power'),
                   col('interfaces.poe_priority').alias('poe_priority'),
                   col('interfaces.poe_class').alias('poe_class'),
                   col('interfaces.poe_mode').alias('poe_mode'),
                   col('interfaces.rx_bytes').alias('rx_bytes'),
                   col('interfaces.tx_bytes').alias('tx_bytes'),
                   col('interfaces.rx_packets').alias('rx_packets'),
                   col('interfaces.tx_packets').alias('tx_packets'),
                   col('interfaces.rx_bps').alias('rx_bps'),
                   col('interfaces.tx_bps').alias('tx_bps'),
                   col('interfaces.rx_errors').alias('rx_errors'),
                   col('interfaces.rx_undersize_errors').alias('rx_undersize_errors'),
                   col('interfaces.rx_oversize_errors').alias('rx_oversize_errors'),
                   col('interfaces.rx_fcserrors').alias('rx_fcserrors'),
                   col('interfaces.rx_overrun_errors').alias('rx_overrun_errors'),
                   col('interfaces.rx_discards').alias('rx_discards'),
                   col('interfaces.tx_errors').alias('tx_errors'),
                   col('interfaces.tx_drops').alias('tx_drops'),
                   col('interfaces.tx_mtuerrors').alias('tx_mtuerrors'),
                   col('interfaces.txcarrier_transition').alias('txcarrier_transition'),
                   col('interfaces.tx_mcast_packets').alias('tx_mcast_packets'),
                   col('interfaces.tx_bcast_packets').alias('tx_bcast_packets'),
                   col('interfaces.rx_mcast_packets').alias('rx_mcast_packets'),
                   col('interfaces.rx_bcast_packets').alias('rx_bcast_packets'),
                   col('interfaces.rx_l3_incompletes').alias('rx_l3_incompletes'),
                   col('interfaces.rx_l2_channel_error').alias('rx_l2_channel_error'),
                   col('interfaces.rx_l2_mismatch_timeouts').alias('rx_l2_mismatch_timeouts'),
                   col('interfaces.rx_fifo_errors').alias('rx_fifo_errors'),
                   col('interfaces.rx_resource_errors').alias('rx_resource_errors'),
                   col('interfaces.auto_negotiation_status').alias('auto_negotiation_status'),
                   col('interfaces.media_type').alias('media_type'))

vp_udf = udf(lambda x: 'phy' if (x[:2]=='ge') | (x[:2]=='xe') | (x[:2]=='et') else 'vir', StringType())
dfin = dfin.withColumn('type', vp_udf('name'))

dfin = dfin.filter(col('media_type')=='copper')

convert_time = udf(lambda x: datetime.utcfromtimestamp(x/1000000).strftime('%Y-%m-%dT%H:%M:%S.%f+00:00')[:16], StringType())
dfin = dfin.withColumn('time', convert_time('when'))

dfin = dfin.filter(dfin.poe_power_limit>0)
pct = udf(lambda x: x[0]/x[1], FloatType())
dfin = dfin.withColumn('power_pct', pct(struct('poe_power','poe_power_limit')))

dfin = dfin.filter(col('delta')==True)

reix = udf(lambda x: int(x[3]), IntegerType())
dfin = dfin.withColumn('re_ix', reix(dfin.name))

dfin.write.parquet('s3://mist-data-science-dev/jing/switch/interfaces'+date)

# Explode dflldp
dflldp = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'when',
                 'remote_addr',
                 'hostname',
                 'model',
                 'firmware_version',
                 'serial_number',
                 'stpbridge_protocol',
                 'stpbridge_root_id',
                 'stpbridge_root_cost',
                 'stpbridge_root_port',
                 'stpbridge_hello_time',
                 'stpbridge_max_age',
                 'stpbridge_forward_delay',
                 'stpbridge_msg_age',
                 'stpbridge_topo_chg_cnt',
                 'stpbridge_last_topo_chg',
                 'stpbridge_bridge_id',
                 'uptime',
                 'stats_delta',
                 'delta',
                 'deltaErrorMsg',
                 'delta_interval',
                 'poe_controller_module_index',
                 'poe_controller_max_power',
                 'poe_controller_consumption',
                 'poe_controller_guardband',
                 explode_outer('lldpneighbors').alias('lldp'))

dflldp = dflldp.select('mac',
                   'when',
                   col('lldp.local_port_id').alias('local_port_id'),
                   col('lldp.local_parent_iface_name').alias('local_parent_iface_name'),
                       col('lldp.remote_chassis_id').alias('remote_chassis_id'),
                       col('lldp.remote_chassis_idsubtype').alias('remote_chassis_idsubtype'),
                       col('lldp.remote_port_desc').alias('remote_port_desc'),
                       col('lldp.remote_system_name').alias('remote_system_name'),
                       col('lldp.remote_mgmt_addr').alias('remote_mgmt_addr'),
                       col('lldp.remote_system_desc').alias('remote_system_desc'),
                       col('lldp.remote_mac').alias('remote_mac'),
                       col('lldp.remote_vlan_id').alias('remote_vlan_id'),
                       col('lldp.remote_hardware').alias('remote_hardware'),
                       col('lldp.remote_serial').alias('remote_serial'),
                       col('lldp.remote_manufacture').alias('remote_manufacture'),
                       col('lldp.remote_system_capability').alias('remote_system_capability'))

# don't need join re_ix as name has handle different chassis
dflldp = dflldp.withColumnRenamed('mac','mac1') \
                .withColumnRenamed('when','when1') 
dfuplink = dfin.join(dflldp, (dfin.mac==dflldp.mac1)&(dfin.when==dflldp.when1)&(dfin.name==dflldp.local_port_id), 'left_outer') \
                .drop('mac1','when1')


# Uplink Model

In [ ]:

# def uplink_lldp(df):
#     if df.select('remote_hardware')[:2] == 'AP':
#         return 0
# #     if df.select('remote_hardware') == '...':
# #         return True
# #     else:
# #         return False
    
# def uplink_speed(df):
#     if df.select('mbps') > 1000: 
#         return True
#     else: 
#         return False

# def uplink_ae(df):
#     if df.select('name') == 'ae':
#         return True
#     else:
#         return False
    
# def uplink_stp(df):
#     if df.select('STPRole') == 'root':
#         return True
#     else: 
#         return False

# def uplink_stproot(dfmac, name):
#     if dfmac.filter(col('stpbridge_root_port')==name).count() > 0:
#         return True
#     else:
#         return False
    
# def uplink_data(df, rxmax, txmax):
#     if df.select('tx_packets') == txmax: 
#         return True
#     elif df.select('rx_packets') == rxmax:
#         return True
#     else:
#         return False

# def uplink(mac, module_index, name, time, df_in):
#     df = df_in.filter(col('mac')==mac) \
#                 .filter(col('name')==name) \
#                 .filter(col('re_ix')==module_index) \
#                 .filter(col('time')==time)
#     dfmac = df_in.filter(col('mac')==mac) \
#                 .filter(col('re_ix')==module_index) \
#                 .filter(col('time')==time)
#     max_rxpkts = dfmac.agg(max_('rx_packets')).collect()[0]['max(rx_packets)'] 
#     max_txpkts = dfmac.agg(max_('tx_packets')).collect()[0]['max(tx_packets)']
#     root = 0
    
#     # link is down, not root for sure
#     if df.filter(col('link')==False):
#         return 0
    
#     # firmly know it is not root
#     if uplink_lldp(df) == 0:
#         return 0
    
#     if uplink_lldp(df):
#         root += 1

#     if uplink_speed(df):
#         root += 1
        
# #     if uplink_ae(df):
# #         root += 1
        
# #     if uplink_stp(df):
# #         root += 1

#     if uplink_stproot(dfmac, name):
#         root += 1
    
#     if uplink_data(df, max_rxpkts, max_txpkts):
#         root += 1
        
#     return root

# def __main__(dfuplink, time):
#     dict_root = {}
#     for each in dfuplink.filter(col('time')==time).select('mac'),'re_ix','name').distinct().collect():
#         root = uplink(mac, re_ix, name, time, dfuplink)
#         dict_root[each] = root
    
#     itemMaxValue = max(dict_root.items(), key=lambda x: x[1]) 
#     listOfKeys = list()
#     for key, value in dict_root.items():
#         if value == itemMaxValue[1]:
#             listOfKeys.append(key)
    
#     return listOfKeys

# if __name__ == "__main__":
#     __main__(dfuplink, time)
#     # __main__(dfuplink, '0c8126c7054d',0, 'ge-0/0/1','2019-11-05T07:19')   

In [ ]:

# def get_root(x):
#     name = x[0]
#     rxpkts = x[1]
#     txpkts = x[2]
#     bridgeroot = x[3][0]
#     remotehw = x[4]
#     mbps = x[5]
#     remotecap = x[6]
#     # stprole = x[7]
#     d_root = {}
    
#     max_rxpkts = np.max(rxpkts)
#     max_txpkts = np.max(txpkts)
        
#     for i in range(len(name)):
        
#         d_root.setdefault(name[i], 0)
        
#         # bridgeport
#         if name[i] == bridgeroot: 
#             d_root[name[i]] += 1
        
#         # data packets
#         if rxpkts[i] == max_rxpkts:
#             d_root[name[i]] += 1
#         elif txpkts[i] == max_txpkts:
#             d_root[name[i]] += 1
        
#         # remote hardware 
#         if remotehw[i][:2] == 'AP':
#             d_root[name[i]] == -100
        
#         # remote capability
#         if remotecap[i] == 'Bridge Router':
#             d_root[name[i]] += 1
        
#         # speed
#         if mbps[i] > 1000:
#             d_root[name[i]] += 1
        
# #         if name[i] == 'ae':
# #             d_root[name[i]] += 1
    
# #         if stprole[i] == 'root':
# #             d_root[name[i]] += 1
    
#     itemMaxValue = max(d_root.items(), key=lambda x: x[1]) 
#     listOfKeys = list()
#     for key, value in d_root.items():
#         if value == itemMaxValue[1]:
#             listOfKeys.append(key)
        
#     return listOfKeys

# def __main__(dfuplink, mac, re_ix, name, time):

#     df = dfuplink.filter(col('mac')==mac) \
#                 .filter(col('re_ix')==module_index) \
#                 .filter(col('time')==time) \
#                 .filter(col('link')==True)

#     df = df.groupby('mac','re_ix','time').agg(collect_list('name').alias('lst_name'), \
#                                              collect_list('rx_packets').alias('lst_rxpkts'), \
#                                               collect_list('tx_packets').alias('lst_txpkts'), \
#                                               collect_list('stpbridge_root_port').alias('lst_bridgeroot'), \
#                                               collect_list('remote_hardware').alias('lst_remotehw'), \
#                                               collect_list('mbps').alias('lst_mbps'))
#     get_root_udf = udf(get_root, ArrayType(StringType()))
#     df = df.withColumn('root', get_root_udf(struct('lst_name','lst_rxpkts','lst_txpkts','lst_bridgeroot','lst_remotehw','lst_mbps')))
    
# if __name__=="__main__":
#     __main__(dfuplink, mac, re_ix, name, time)

In [ ]:
# Get each root
def bridge(x):
    if x[0] == x[1]: return 1
    else: return 0

def pkts(x): 
    import copy
    rx = x[0]
    tx = x[1]
    port = x[2]
    root = set()
    for i in range(len(rx)):
        temp = copy.deepcopy(tx)
        temp.pop(i)
        if all(rx[i] > t for t in temp):
            root.add(port[i])
    for i in range(len(rx)):
        temp = copy.deepcopy(rx)
        temp.pop(i)
        if all(tx[i] > t for t in temp):
            root.add(port[i])        
    return list(root)
    
def remotehw(x):
    if x[:2] == 'AP': return 0
    else: return 1

def remotecap(x):
    if x == 'Bridge Router': return 1
    else: return 0
    
def mbps(x):
    if x > 1000: return 1
    else: return 0

bridge_udf = udf(bridge, IntegerType())
pkts_udf = udf(pkts, ArrayType(StringType()))
remotehw_udf = udf(remotehw, IntegerType())
remotecap_udf = udf(remotecap, IntegerType())
mbps_udf = udf(mbps, IntegerType())

# Get root from pkts logic
dfpkts = dfuplink.groupby('mac','re_ix','time').agg(collect_list('rx_packets').alias('lst_rxpkts'), \
                                                    collect_list('tx_packets').alias('lst_txpkts'), 
                                                    collect_list('name').alias('lst_port'))
dfpkts = dfpkts.withColumn('rootpkts', pkts_udf(struct('lst_rxpkts','lst_txpkts','lst_port'))) \
                .withColumnRenamed('mac','mac1') \
                .withColumnRenamed('re_ix','re_ix1') \
                .withColumnRenamed('time','time1') \
                .drop('lst_txpkts','lst_rxpkts','lst_port')

dfpkts = dfpkts.select('*', explode('rootpkts').alias('pkts1')).drop('rootpkts')
                           
dffull = dfuplink.join(dfpkts, (dfuplink.mac==dfpkts.mac1)&(dfuplink.re_ix==dfpkts.re_ix1)&(dfuplink.time==dfpkts.time1), 'left') \
                    .drop('mac1','re_ix1','time1')

rootpkts = udf(lambda x: 1 if x[0]==x[1] else 0, IntegerType())  
dffull = dffull.withColumn('pkts', rootpkts(struct('name','pkts1')))  

# Get root from other logics, filter out remotehw = AP
dffull = dffull.fillna(' ')
dffull = dffull.filter(dffull.remotehw==1)
dfroot = dffull.withColumn('remotecap', remotecap_udf('remote_system_capability')) \
                .withColumn('rmbps', mbps_udf('mbps')) \
                .withColumn('remotehw', remotehw_udf('remote_hardware')) \
                .withColumn('bridge', bridge_udf(struct('stpbridge_root_port','name'))) \
                .select('mac','site_id','time','re_ix','name','remotecap','rmbps','bridge','pkts')

# =========================================
# Modeling
pdf = dfroot.select('mac','re_ix','name','remotecap','rmbps','bridge','pkts').distinct().toPandas()
          
# Labeling: 
# 0c8126c71290: ge-0/0/1 0
# 1c9c8cba2e7f: ge-0/0/1 0
# 1c9c8cba2d44: ge-0/0/4 0
# 1c9c8cba2d44: ge-1/0/5 1
# 0256: ge-0/0/0 ?
# 80acacf2f530: ge-0/0/0 0
# 3c8c939495ac: ge-0/0/8 0
# 0c8126c70665: ge-0/0/10 0
# d8b12288af4f: ge-0/0/0 0
# 1c9c8cba2970: ge-0/0/4 and ge-0/0/5 0

label1 = [
    ['1c8126c71290','ge-0/0/1',0],
    ['1c9c8cba2e7f','ge-0/0/1',0],
    ['1c9c8cba2d44','ge-0/0/4',0],
    ['1c9c8cba2d44','ge-1/0/5',1],
    ['80acacf2f530','ge-0/0/0',0],
    ['3c8c939495ac','ge-0/0/8',0],
    ['0c8126c70665','ge-0/0/10',0],
    ['d8b12288af4f','ge-0/0/0',0],
    ['1c9c8cba2970','ge-0/0/4',0],
    ['1c9c8cba2970','ge-0/0/5',0]
]

# Get the mac which has labels
flag = True
for each in label1:
    if flag:
        pdf1 = pdf[(pdf.mac==each[0])&(pdf.re_ix==each[2])]
        flag = False
    else:
        pdf0 = pdf[(pdf.mac==each[0])&(pdf.re_ix==each[2])]
        pdf1 = pd.concat([pdf0, pdf1], axis=0)

# def get_label(mac, re_ix, name):
#     flag = 0
#     for each in label1:    
#         if (mac==each[0])&(name==each[1])&(re_ix==each[2]):
#             flag = 1
#     return flag
     
# pdf1['label'] = pdf1.apply(lambda r: get_label(r.mac, r.re_ix, r.name), axis=1)
                                   
for ix, row in pdf1.iterrows():
    for each in label1:
        if (row['mac']==each[0])&(row['name']==each[1])&(row['re_ix']==each[2]):
            pdf1.at[ix,'label'] = 1
pdf1 = pdf1.fillna(0)

# Logistic Regression 
from sklearn.linear_model import LogisticRegression
X = pdf1[['remotecap','rmbps','bridge','pkts']]
y = pdf1['label']
model = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial')
clf = model.fit(X, y)
clf.score(X, y)  #0.87
clf.predict(X)

In [157]: model.coef_                                                                                                                          
Out[157]: array([[1.01196051, 0.        , 1.37810952, 0.52740574]])
In [158]: model.intercept_                                                                                                                     
Out[158]: array([-1.38771024])




# base = dfroot.select('mac','re_ix','name','time').distinct()
# dfroot1 = dfroot.filter(col('remotehw')==1) \
#                 .filter(col('maxrxpkts')>0) \
#                 .filter(col('maxtxpkts')>0) \
#                 .filter((col('remotecap')>0)|(col('bridge')>0)|(col('pkts')>0)|(col('rmbps')>0)) \
#                 .select('mac','re_ix','name','time','remotecap','bridge','pkts','rmbps') \
#                 .withColumnRenamed('mac','mac1') \
#                 .withColumnRenamed('re_ix','re_ix1') \
#                 .withColumnRenamed('name','name1') \
#                 .withColumnRenamed('time','time1')


# dfjoin = base.join(dfroot1, (base.mac==dfroot1.mac1)&(base.re_ix==dfroot1.re_ix1)&(base.name==dfroot1.name1)&(base.time==dfroot1.time1), 'left') \
#                 .drop('mac1','re_ix1','name1','time1')

# dfjoin = dfjoin.fillna(0)

# sumudf = udf(lambda x: x[0]+x[1]+x[2]+x[3], IntegerType())
# df0 = dfjoin.withColumn('root', sumudf(struct('remotecap','bridge','pkts','rmbps')))
# df1 = df0.groupby('mac','re_ix','name').agg(sum_('root').alias('sumroot'))
# df2 = df1.filter(df1.sumroot>0)
# df2 = df2.withColumnRenamed('mac','mac1') \
#             .withColumnRenamed('re_ix','re_ix1') \
#             .withColumnRenamed('name','name1') 

# dffinal = dfjoin.join(df2, (dfjoin.mac==df2.mac1)&(dfjoin.re_ix==df2.re_ix1)&(dfjoin.name==df2.name1), 'inner') \
#                 .drop('mac1','re_ix1','name1','sumroot')

# dffinal.toPandas().to_csv('uplink.csv')                          


In [ ]:
# # Old logic for pkts
# # Get each root
# def bridge(x):
#     if x[0] == x[1]: return 1
#     else: return 0

# def pkts(x): 
#     if x[0] == x[2]: return 1
#     elif x[1] == x[3]: return 1
#     else: return 0
    
# def remotehw(x):
#     if x[:2] == 'AP': return 0
#     else: return 1

# def remotecap(x):
#     if x == 'Bridge Router': return 1
#     else: return 0
    
# def mbps(x):
#     if x > 1000: return 1
#     else: return 0

# bridge_udf = udf(bridge, IntegerType())
# pkts_udf = udf(pkts, IntegerType())
# remotehw_udf = udf(remotehw, IntegerType())
# remotecap_udf = udf(remotecap, IntegerType())
# mbps_udf = udf(mbps, IntegerType())

# dfmac = dfuplink.groupby('mac','re_ix','time').agg(max_('rx_packets').alias('maxrxpkts'), max_('tx_packets').alias('maxtxpkts')) \
#                 .withColumnRenamed('mac','mac1') \
#                 .withColumnRenamed('re_ix','re_ix1') \
#                 .withColumnRenamed('time','time1')
# dffull = dfuplink.join(dfmac, (dfuplink.mac==dfmac.mac1)&(dfuplink.re_ix==dfmac.re_ix1)&(dfuplink.time==dfmac.time1), 'left') \
#                     .drop('mac1','re_ix1','time1')

# dffull = dffull.fillna(' ')
# dfroot = dffull.withColumn('remotecap', remotecap_udf('remote_system_capability')) \
#                 .withColumn('rmbps', mbps_udf('mbps')) \
#                 .withColumn('remotehw', remotehw_udf('remote_hardware')) \
#                 .withColumn('bridge', bridge_udf(struct('stpbridge_root_port','name'))) \
#                 .withColumn('pkts', pkts_udf(struct('rx_packets','maxrxpkts','tx_packets','maxtxpkts')))

# base = dfroot.select('mac','re_ix','name','time').distinct()
# dfroot1 = dfroot.filter(col('remotehw')==1) \
#                 .filter(col('maxrxpkts')>0) \
#                 .filter(col('maxtxpkts')>0) \
#                 .filter((col('remotecap')>0)|(col('bridge')>0)|(col('pkts')>0)|(col('rmbps')>0)) \
#                 .select('mac','re_ix','name','time','remotecap','bridge','pkts','rmbps') \
#                 .withColumnRenamed('mac','mac1') \
#                 .withColumnRenamed('re_ix','re_ix1') \
#                 .withColumnRenamed('name','name1') \
#                 .withColumnRenamed('time','time1')


# dfjoin = base.join(dfroot1, (base.mac==dfroot1.mac1)&(base.re_ix==dfroot1.re_ix1)&(base.name==dfroot1.name1)&(base.time==dfroot1.time1), 'left') \
#                 .drop('mac1','re_ix1','name1','time1')

# dfjoin = dfjoin.fillna(0)

# sumudf = udf(lambda x: x[0]+x[1]+x[2]+x[3], IntegerType())
# df0 = dfjoin.withColumn('root', sumudf(struct('remotecap','bridge','pkts','rmbps')))
# df1 = df0.groupby('mac','re_ix','name').agg(sum_('root').alias('sumroot'))
# df2 = df1.filter(df1.sumroot>0)
# df2 = df2.withColumnRenamed('mac','mac1') \
#             .withColumnRenamed('re_ix','re_ix1') \
#             .withColumnRenamed('name','name1') 

# dffinal = dfjoin.join(df2, (dfjoin.mac==df2.mac1)&(dfjoin.re_ix==df2.re_ix1)&(dfjoin.name==df2.name1), 'inner') \
#                 .drop('mac1','re_ix1','name1','sumroot')

# dffinal.toPandas().to_csv('uplink.csv')                          


# --END--

-----------------------